<a href="https://colab.research.google.com/github/veillette/jupyterNotebooks/blob/main/Optics/bestFitLineAndSnellsLaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import numpy as np
import numpy as np

# Import non-linear least squares to fit a function, f, to data.
from scipy.optimize import curve_fit

# Library to plot data
import matplotlib.pyplot as plt

In [ ]:
def fitRelation( x, n):
    return n * x

In [ ]:
# Generate some data points
angles_of_incidence = np.linspace(0, np.pi/2, 100)
sines_of_incidence = np.sin(angles_of_incidence)
sines_of_refraction = 1.2 * sines_of_incidence

In [ ]:
# Sample data
angles_of_incidence = np.array([0,10,20,30,40,50])
angles_of_refraction = np.array([0,13,25,35,45,58])
sines_of_incidence = np.sin(angles_of_incidence*np.pi/180)
sines_of_refraction = np.sin(angles_of_refraction*np.pi/180)


In [ ]:
# Perform the weighted least squares fit
params, covariance = curve_fit(fitRelation, sines_of_incidence, sines_of_refraction)

# Extract the optimized parameters
n_opt = params

# Calculate the standard deviation of the parameters
n_opt_std_dev = np.sqrt(np.diag(covariance))

In [ ]:
# Display the optimized parameters and their standard deviations
print("Optimized parameters:")
print(f"n = {n_opt} ± {n_opt_std_dev}")


In [ ]:
# Plot the data
plt.plot(sines_of_incidence, sines_of_refraction, 'o')

# the best fit line
angles_of_incidence = np.linspace(0, 80, 100)
x = np.sin(angles_of_incidence*np.pi/180)
y = fitRelation(x, n_opt)
plt.plot(x, y)

# Label the axes and show the plot
plt.xlabel('Sine of Incidence Angle')
plt.ylabel('Sine of Refracted Angle')
plt.title('Best Fit Line for Snell\'s Law')
plt.show()